In [ ]:
#coding:gbk

import cx_Oracle, os, csv
import pandas as pd
import numpy as np
from pandas import DataFrame
conn = cx_Oracle.connect(' ',' ',' ')  # sql info

def _exec_sql(sql):
    cursor = conn.cursor()
    cursor.execute(sql)
    conn.commit()
        
def File_Handler(begDate, endDate, compid, traderid):
    path = "/datas/share/AnalyStkTick/yulintest/from_%s_to_%s/" %(begDate,endDate)
    All_files = os.listdir(path)
    df = pd.read_excel(path + "%s-%s-%s_BENCHONLY-93000-150000-%s.xlsx" %(begDate,endDate,compid,traderid))
    for i in range(len(All_files)):
        if (All_files[i][0:8] == begDate) and (All_files[i][21] == '-'):
            df_temp = pd.read_excel(path + All_files[i], sheet_name = u'profitdetail' )
            df = pd.concat([df,df_temp])
    return df


def Filter_Odds(begDate, endDate, compid, traderid):
    df = File_Handler(begDate, endDate, compid, traderid)
    os.system("mkdir -p check0121")
    os.chdir("check0121")
    os.system("mkdir -p from_%s_to_%s" %(begDate,endDate))
    os.chdir("..")
    headers = ['accountid', 'dte', 'tcurve_profit(e4)', 'IMPACTBENCH(e4)', 'PUNISH(e4)']
    logname = open("/datas/share/AnalyStkTick/yulintest/check0121/from_%s_to_%s/check0121.csv" %(begDate,endDate) , 'w', newline='')
    log = csv.DictWriter(logname, headers)
    log.writeheader()
    for i in range(df.shape[0]):
        p1 = df.values[i][np.where(df.columns == '成交冲击成本(万)')][0]
        p2 = df.values[i][np.where(df.columns == '基准冲击成本(万)')][0]
        p3 = df.values[i][np.where(df.columns == '惩罚(万)')][0]
        if abs(p1 - p2) > abs(0.1 * p2):
            log.writerow({'accountid':df.values[i][np.where(df.columns == '产品')][0], 'dte':df.values[i][np.where(df.columns == '日期')][0], 'tcurve_profit(e4)':p1, 'IMPACTBENCH(e4)':p2})
        if p1 == p2 == 0:
            log.writerow({'accountid':df.values[i][np.where(df.columns == '产品')][0], 'dte':df.values[i][np.where(df.columns == '日期')][0], 'tcurve_profit(e4)':p1, 'IMPACTBENCH(e4)':p2})
        if p3 <= -1:
            log.writerow({'accountid':df.values[i][np.where(df.columns == '产品')][0], 'dte':df.values[i][np.where(df.columns == '日期')][0], 'PUNISH(e4)':p3})
    

def Update_SQL(new_startTME, new_endTME, new_accountid, new_dte, traderid):
    updatesql = " update instructs_algo set STARTTME={}, ENDTME={} where accountid like '%{}%_F' and dte={} and traderid={}".format(new_startTME,new_endTME,new_accountid,new_dte,traderid)
#     testupdatesql = " select * from instructs_algo where accountid like '%{}%_{}' and dte={} and traderid={}".format(new_accountid,SoF,new_dte,traderid)
#     df_test = pd.read_sql(testupdatesql,conn)
#     print(df_test)
    _exec_sql(testupdatesql)
    
    
    
def Trade_Time(begDate, endDate, compid, traderid):
    
    Filter_Odds(begDate, endDate, compid, traderid)
    
    path = "/datas/share/AnalyStkTick/yulintest/check0121/from_%s_to_%s/check0121.csv" %(begDate,endDate)
    df_All = pd.read_csv(path, sep=',')
    headers = ['accountid', 'dte', 'start_actTME', 'end_actTME', 'startTME', 'endTME', 'start_actTME_s', 'end_actTME_s', 'startTME_s', 'endTME_s']
    logname = open("/datas/share/AnalyStkTick/yulintest/check0121/from_%s_to_%s/TradeTime.csv" %(begDate,endDate) , 'w', newline='')
    log = csv.DictWriter(logname, headers)
    log.writeheader()
    for i in range(df_All.values.shape[0]):
        #print(i)
        accountid, SoF = df_All.values[i][0].split('_')
        date = df_All.values[i][1]
        # 指令
        sql = " select DTE,ACCOUNTID,STARTTME,ENDTME from instructs_algo where dte={} and accountid like '%{}%{}%'".format(date,accountid, SoF)
        df_instr = pd.read_sql(sql,conn)
        # 实际
        sql = " select DTE,ACCOUNTID,TRADETME from transactions_algo where dte={} and mod(qty,100) =0 and accountid like '%{}%{}%' and tradetype=1 and traderid=102".format(date,accountid,SoF)
        df_trans = pd.read_sql(sql,conn)
        startTME = np.min(df_instr['STARTTME'])
        endTME = np.max(df_instr['ENDTME'])
        actTME = []
        for j in range(df_trans.shape[0]):
            temp = str(df_trans['TRADETME'][j])
            temp = temp[len(temp)-8:len(temp)].split(':')
            temp = int(temp[0]+temp[1]+temp[2])
            actTME.append(temp)
        start_actTME = np.min(actTME)
        end_actTME = np.max(actTME)
        start_actTME_s = 3600*int(start_actTME/10000) + 60*int((start_actTME%10000)/100) + (start_actTME%100)
        end_actTME_s = 3600*int(end_actTME/10000) + 60*int((end_actTME%10000)/100) + (end_actTME%100)
        startTME_s = 3600*int(startTME/10000) + 60*int((startTME%10000)/100) + (startTME%100)
        endTME_s =3600*int(endTME/10000) + 60*int((endTME%10000)/100) + (endTME%100)
        if (abs(startTME_s - start_actTME_s)> 300) or (abs(end_actTME_s - endTME_s)> 300):
            log.writerow({'accountid':df_trans['ACCOUNTID'][j], 'dte':df_trans['DTE'][j], 'start_actTME':str(start_actTME), 'end_actTME':str(end_actTME), 'startTME':str(startTME), 'endTME':str(endTME), 'start_actTME_s':str(start_actTME_s), 'end_actTME_s':str(end_actTME_s), 'startTME_s':str(startTME_s), 'endTME_s':str(endTME_s)})
            # 修改_F指令时间
            if SoF == 'F':
                new_startTME = int(start_actTME/100)*100
                new_endTME = int(end_actTME/100)*100
                print(new_startTME)
                print(new_endTME)
                new_accountid = accountid
                new_dte = df_trans['DTE'][j]
                Update_SQL(new_startTME, new_endTME, new_accountid, new_dte, traderid)
                print('%s %s updated' %(new_accountid,new_dte))
                
                
    
if __name__ == '__main__':
    if len(sys.argv) < 4:
        print >>sys.stderr, 'usage:%s begDate endDate compId, traderId'%sys.argv[0]  # 查某一支特定产品
        sys.exit()

    begDate = str(int(sys.argv[1]))
    endDate = str(int(sys.argv[2]))
    compid   = str(sys.argv[3])
    traderid = None if len(sys.argv) < 5 else str(int(sys.argv[4]))
    
    Trade_Time(begDate, endDate, compid, traderid)